Refernce implementation:

https://github.com/openai/spinningup/blob/master/spinup/algos/tf1/trpo/core.py

Video lectures:

https://www.youtube.com/watch?v=ycCtmp4hcUs&list=PLkFD6_40KJIznC9CDbVTjAF2oyt8_VAe3&index=14

https://www.youtube.com/watch?v=KjWF8VIMGiY

# Trust Region Policy Optimization (TRPO)

We can measure performance of a policy $\pi$ by empirical estimate
$$J(\pi) = \mathbb{E}_{\tau \sim \pi}\left[R(\tau)\right] = \mathbb{E}\left[\sum_{t} \gamma^t r(s_t)\right]$$
Similarly to "Vanilla" policy gradient we can introduce an advantage function- a cumulative future reward relative to a baseline.

> (Comment) Formally, since we observe trajectories where agent, at some time step t, at some state $s_t$, only takes a single action $a_t$ out of all possible actions, future cumulative reward we observe is action-value function $Q(s_t,a_t)$. Agent may take different actions from that state. On average each state will have expected future reward $V(s_t)$

So we define advantage function, for a policy $\pi$, as
$$A_\pi(s,a) = Q_\pi(s,a)-V_\pi(s)$$

If we want to compare two policies we can use following expression
$$J(\pi') - J(\pi) = \mathbb{E}_{\tau \sim \pi'}\left[\sum_{t} \gamma^t A_\pi(s_t,a_t)\right]$$
Notice that trajectory (states and actions) are sampled from new policy $\pi'$, while advantage is measured using old policy $\pi$.

> __Proof:__
>
> This expression can be shown to be true via explicitly writing out definitions for $Q$
> $$ Q_\pi(s_t,a_t) = \underbrace{R(s_t,a_t,s_{t+1})}_{\text{Immediate reward for taking } a_t} + \gamma \underbrace{V_\pi(s_{t+1})}_{\text{Future reward from next state}}$$
> $$J(\pi') - J(\pi) = \mathbb{E}_{\tau \sim \pi'} \bigg[\sum_{t=0} \gamma^t \big( R(s_t,a_t,s_{t+1}) + \gamma  V_\pi(s_{t+1}) - V_\pi(s_{t}) \big)\bigg]$$
> $$ = \mathbb{E}_{\tau \sim \pi'} \bigg[\sum_{t=0}  \gamma^t  R(s_t,a_t,s_{t+1})\bigg] +  \mathbb{E}_{\tau \sim \pi'} \bigg[\sum_{t=0} \gamma^{t+1}  V_\pi(s_{t+1})\bigg] - \mathbb{E}_{\tau \sim \pi'}\bigg[\sum_{t=0} \gamma^{t}  V_\pi(s_{t})\bigg] $$
> $$ = J(\pi') + \mathbb{E}_{\tau \sim \pi'} \bigg[\sum_{t=1} \gamma^{t}  V_\pi(s_{t+1})  - \sum_{t=0} \gamma^{t}  V_\pi(s_{t})\bigg]$$
> where for second term we have renamed summation index. 
> We can see that 
> $$\sum_{t=0} \gamma^{t}  V_\pi(s_{t})= V_\pi(s_{0}) +  \sum_{t=1} \gamma^{t}  V_\pi(s_{t}) $$
> so
> $$\sum_{t=1} \gamma^{t}  V_\pi(s_{t+1}) - \sum_{t=0} \gamma^{t}  V_\pi(s_{t}) =  V_\pi(s_{0})$$
> Thus
> $$J(\pi') - J(\pi) = J(\pi')- \mathbb{E}_{\tau \sim \pi'} \bigg[V_\pi(s_{0})\bigg] $$
> $V_\pi(s_t)$ is defined as expected cumulative reward from time $t$ : $\{t, t+1,\dots, t+k\}$
> $$V_\pi(s_t) = \mathbb{E}_{\tau \sim \pi} \left[\sum_k \gamma^k r(s_{t+k}) \right]$$
> so for $t=0$ (rename $k$ to $t$)
> $$V_\pi(s_0) = \mathbb{E}_{\tau \sim \pi} \left[\sum_t \gamma^t r(s_{t}) \right] \longleftrightarrow J(\pi)$$
> which does not change under expectation. Proof complete.

# Policy improvement metric

We can define policy improvement as a metric
$$L_\pi(\pi') = J(\pi') - J(\pi) = \mathbb{E}_{\tau \sim \pi'}\left[\sum_{t} \gamma^t A_\pi(s_t,a_t)\right]$$
While we have trajectory data collected using old policy $\pi$, these are not trajectories that can estimated over for new policy $\pi'$

Logic step is to change the expectation to be taken over old policy via _importance sampling_. 

This step is not straightforward. In paper authors transition to point of view of (discounted) visitation frequencies
$$d_{\pi}(s) = \underbrace{1-\gamma}_{\text{normalization}}\sum_{t = 0} \gamma^t P_\pi(s_t = s)$$
> (Side note) IMO this approach is easy to understand if we ask what is the performance based on 'average' state value:
> $$J(\pi) = \sum_s P_\pi(s_t = s) V_\pi(s) = \sum_s P_\pi(s_t = s) \sum_a \pi(a|s) Q_\pi(a,s)$$
and rewrite expectation as sum over states
$$J(\pi') - J(\pi) = \mathbb{E}_{\tau \sim \pi'}\left[\sum_{t} \gamma^t A_\pi(s_t,a_t)\right]=\sum_{t = 0} \sum_{s} P_{\pi'}(s_t = s)\sum_a \pi'(a|s) \gamma^t A_{\pi}(s,a)$$
$$ = \frac{1}{1-\gamma}\sum_s d_{\pi'}(s)\sum_a \pi'(a|s) A_{\pi}(s,a)= 
\frac{1}{1-\gamma}\mathbb{E}_{a \sim \pi', s \sim d_{\pi'}} \bigg[A_{\pi}(s,a)\bigg]$$
We can introduce importance sampling here so $a \sim \pi$
$$J(\pi') - J(\pi) = 
\frac{1}{1-\gamma}\mathbb{E}_{\boxed{a \sim \pi}, s \sim d_{\pi'}} \bigg[\frac{\pi'(a|s)}{\pi(a|s)}A_{\pi}(s,a)\bigg]$$

We see that state visitation frequency $d_{\pi'}$ is still according to new policy.

Approximation is such, that we consider $d_{\pi'} \approx d_{\pi}$
>(NOTE) Formally I would:
>*  introduce importance sampling ratio $\frac{d_{\pi'}}{d_{\pi}}$;
>*  change to expectation over $s\sim d_{\pi}$
>*  set ratio to $1$.
$$J(\pi') - J(\pi) \approx 
\frac{1}{1-\gamma}\mathbb{E}_{a \sim \pi, \boxed{s \sim d_{\pi}}} \bigg[\frac{\pi'(a|s)}{\pi(a|s)}A_{\pi}(s,a)\bigg]$$
$$ = \frac{1}{1-\gamma}\sum_s d_{\pi}(s)\sum_a \frac{\pi'(a|s)}{\pi(a|s)} A_{\pi}(s,a)$$
$$= \sum_{t = 0} \sum_{s} P_\pi(s_t = s|\pi)\sum_a \frac{\pi'(a|s)}{\pi(a|s)} \gamma^t A_{\pi}(s,a)$$
returning from point of view of visitation frequency we get approximation
$$\boxed{J(\pi') - J(\pi) = \mathbb{E}_{\tau \sim \pi}\left[\sum_{t} \gamma^t \frac{\pi'(a|s)}{\pi(a|s)} A_\pi(s_t,a_t)\right]}$$
__Approximation holds if policies are similar enough $d_{\pi'} \approx d_{\pi}$ to frequent same states__

_That would mean that all probability distributions over actions (for all particular states) should be similar 'enough'_
$$\forall s: \pi'(\cdot|s) \approx \pi(\cdot|s)$$

## Milestone #1: objective function

Because we will parameterize our policy, we are transitioning from notation of $\pi$ and $\pi'$ to parameters $\theta$ and $\theta_{old}$ or $\theta_{t+1}$ and $\theta_{t}$, depending on context

We can define an objective function
$$L_{\theta_{old}}(\theta) = J(\theta) - J(\theta_{old})$$

In order to improve $J(\theta)$, we will conduct gradient ascent using update rule
$$\theta_{t+1} = \theta_t + \alpha \nabla_{\theta} L_{\theta_{t}}(\theta)$$

Technically term $J(\theta_{old})$ is constant w.r.t parameters $\theta$, so we can drop it (i hope?). We also disable discounting $\gamma = 1$.

Following expression is called __surrogate reward__

$$\boxed{L_{\theta_{old}}(\theta) =  \mathbb{E}_{\tau \sim \pi_{old}}\left[\sum_{t}  \frac{\pi(a_t|s_t)}{\pi_{old}(a_t|s_t)} A_{\pi_{old}}(s_t,a_t)\right]}$$
We have defined an objective function


## Milestone #2. Gradient $\vec{g}$

Due to linearity of expectation and that $\pi_{old}(a|s)$ is independent of $\theta$
$$\vec{g} = \nabla_\theta L_{\theta_{old}}(\theta) =  \mathbb{E}_{\tau \sim \pi}\left[\sum_{t} \gamma^t \frac{\nabla_\theta  \pi(a_t|s_t)}{\pi_{old}(a_t|s_t)} A_{\pi_{old}}(s_t,a_t)\right]$$
_We wont actually need it in implementation, because it will be computed via autodifferentiation._

> (Questionable)<br>
>Notice that if $\theta = \theta_{old}$ (not sure if i can change $\nabla$ haha)
>$$\nabla_\theta L_{\theta_{old}}(\theta)\big|_{\theta = \theta_{old}} = \mathbb{E}_{\tau \sim \pi}\left[\sum_{t} \gamma^t \frac{\nabla_{\theta_{old}}  \pi_{old}(a_t|s_t)}{\pi_{old}(a_t|s_t)} A_{\pi_{old}}(s_t,a_t)\right] = \mathbb{E}_{\tau \sim \pi}\left[\sum_{t} \gamma^t \nabla_{\theta_{old}}\log  \pi_{old}(a_t|s_t) A_{\pi_{old}}(s_t,a_t)\right] = \nabla_{\theta_{old}}J(\theta_{old})$$
>which is a 1st order term in expansion of 
>$$J(\theta_{old}+ d\theta) = J(\theta_{old}) + \nabla_{\theta_{old}}J(\theta_{old}) d\theta$$
>_Significance? IDK. You tell me!_

## Milestone #3. Policy update (Overview)

Procedure of constructing gradient ascent update rule is identical to one Natural Policy Gradient (NPG) method [02_Natural_Policy_Gradient.ipynb](02_Natural_Policy_Gradient.ipynb). 

Procedure is the following, we:
1.  Create an unconstrained optimization objective where we implement KL divergence penalty. 

    $$L_{\theta_{old}}(\theta + \delta) \approx J(\theta)\big|_{\theta = \theta_{old}}  - J(\theta_{old}) + \nabla_\theta J(\theta)^T\big|_{\theta = \theta_{old}}\delta$$
    $$ D_{KL}\big(\pi(\theta)||\pi(\theta+\delta)\big) \approx \frac{1}{2} \delta^T \nabla^2 D_{KL} \delta = \frac{1}{2} \delta^T H \delta$$
    1st order in loss and 2nd order in KL div
    $$\delta^* = \argmax_\delta \nabla_\theta J(\theta)^T\big|_{\theta = \theta_{old}}\delta - \lambda ( \delta^T \nabla^2 D_{KL} \delta - \epsilon)$$
    
2.  Find appropriate ascend direction $H^{-1}\vec{g}$;
    $$ \delta^* = \frac{2}{\lambda} H^{-1}\nabla_\theta J(\theta) = \frac{2}{\lambda} H^{-1}\vec{g}$$
3.  Find step length $\alpha$;
4.  (NEW TO TRPO) Perform backtracking line search to satisfy KL divergence $\alpha_j$

## KL divergence constraint

$$D_{KL}(\pi||\pi_{old}) \leq \epsilon$$
_Might better to write it for all possible actions $\pi(\cdot,s)$ and we want to limit maximal divergence encountered in any state $s$
$$ \max_s D_{KL}(\pi(\cdot,s)|| \pi_{old}(\cdot,s)) \leq \epsilon$$
_But in practice we might take __average__ over all $s$ instead_

## NPG Update Recap

> In __NPG__ we used quadratic approximation to KL divergence (and we will do it again) in form of Fisher Information Matrix (FIM) ([Fisher_Information.ipynb](../../Statistics/Fisher_Information.ipynb)) which reused 
> $$\nabla_{\theta} \ log \ \pi_\theta(a_t|s_t)$$
> needed for a gradient.
> $$F =\nabla^2 D_{KL}= \mathbb{E}\bigg[\nabla_\theta \log \pi(x; \theta) \ \nabla_\theta \log \pi(x; \theta)^T\bigg]$$

## TRPO Update

In __TRPO__ we dont not implement FIM, but compute Hessian of KL divergence ([KL_Divergence.ipynb](../../Statistics/KL_Divergence.ipynb))
$$\boxed{D_{KL}\bigg(p(x; \theta) ||p(x; \theta + \delta) \bigg) \overset{\text{2nd order}}{\approx} \frac{1}{2}\delta^T \underbrace{H}_{\nabla^2 D_{KL}} \delta = \frac{1}{2}(\theta - \theta_{old})^T H (\theta - \theta_{old}) }$$
But this requires us to actually calculate $D_{KL}$. We will see how later.
***

## Policy update expression #1

This bring us to policy parameter update rule. Here we use Hessian instead of FIM. 
$$\theta_{t+1} = \theta_t + \sqrt{\frac{2\epsilon}{\vec{g}^T {H^{-1}} \vec{g}}} H^{-1}\vec{g}$$

## Milestone #4. Conjugate Gradient

Term $H^{-1}\vec{g}$ can be calculated (or approximated) via Conjugate Gradient (CG) method ([Notes_Method_of_Conjugate_Gradient_Descent.ipynb](../../optimization/Notes_Method_of_Conjugate_Gradient_Descent.ipynb)), which iteratively solves problem 
$$H \vec{x} = \vec{g} \longrightarrow \vec{x}^* = H^{-1} \vec{g}$$
Part of CG iterations is repeated calculation of matrix-vector (descent direction) multiplication $H \vec{v}$ (in algorithm: $A\vec{d}_i/A\vec{x}_i$).

> In fact these are the only times $H$ is used
*   $$\vec{r}_i = \vec{b}_i - A \vec{x}_i \ \text{ (Neg. gradient)}$$
*   $$\alpha_i = \frac{\vec{r}_i^T\vec{r}_i}{\vec{d}_i^T A \vec{d}_i} \ \text{ (Line search step size)}$$
*   $$\vec{r}_{i+1} = \vec{r}_i - \alpha_i A\vec{d}_i \ \text{ (Beginning of this notebook)}$$

Because our problem $H \vec{x} = \vec{g}$ involves a Hessian matrix, we can do major simplifications.

We rewrite it as a Jacobian of a gradient
$$H(\cdot)= \underbrace{J}_{\text{Jacobian}}\nabla (\cdot)$$
which allows us to rewrite Hessian-vector product using only gradients
$$H \vec{v} = J\big(\nabla D_{KL}\big) \vec{v} = J\big(\nabla D_{KL} \cdot \vec{v}\big) $$
This to so called 'Hessian-vector product trick' ([CG_Hessian_vector_trick.ipynb](../../optimization/CG_Hessian_vector_trick.ipynb))

In practice that means that we define a function $$H_v(\vec{v}) = J\big(\nabla D_{KL} \cdot \vec{v}\big)$$
where $\nabla D_{KL}$ is precomputed and outer Jacobian 'remembers' computational graph to make repeated calculations fast.



## Policy update expression #2 (using CG)

If we use CG approach where $\vec{x}^* = H^{-1} \vec{g}$, we can rewrite update
$$\theta_{t+1} = \theta_t + \sqrt{\frac{2\epsilon}{\vec{g}^T {H^{-1}} \vec{g}}} H^{-1}\vec{g}$$
as
$$\boxed{\vec{\theta}_{t+1} = \vec{\theta_t} + \sqrt{\frac{2\epsilon}{\vec{g}^T \vec{x}^*}} \vec{x}^*}$$
since $\vec{g}^T = \vec{x}^{*T}H^T = \vec{x}^{*T}H$ you might see update written as
$$\boxed{\vec{\theta}_{t+1} = \vec{\theta_t} + \sqrt{\frac{2\epsilon}{\vec{x}^{*T}H \vec{x}^*}} \vec{x}^*}$$
Now we know how/that we can calculate
$$\boxed{H_v(\vec{v}) = J\big(\nabla D_{KL} \cdot \vec{v}\big)}$$
__But how do we compute $D_{KL}$ to begin with?__

# Milestone #5a: Policy parametrization

>Reminder:<br>Simple environments with small number of observation space can be dealt with by tabular approach. When observation space becomes large/continuous, we have to use approximations that generalize. Common case is to use neural networks (NNs), in simplest case Multi Layer Perceptron.

Depending on whether action space is discrete or continuous, we will have a different approach on how we build NN and produce actions.

Our goal is to calculate for a policy:
1. probability $\pi(a_t|s_t)$
2. KL divergence $D_{KL}$
***
***

## Discrete actions (easy)

> Here we will consider that agent can take only one action from list on $N_{act}$ actions at a time. 

> In discrete action case actions are simply integers $[0,1,2,\dots]$, an abstraction for what they mean in environment [North, East, North-East,...].

#### Definition of NN

For a policy implemented via NN we want to feed in a state (a list of states) and for NN to produce list of probabilities for each action in that state (for each state in a list).
$$\vec{\pi }(s)= \{\pi(a_1|s),\dots,\pi(a_{N_{act}}|s)\}^T$$
Setting NN to produce probabilities directly is difficult, because $\vec{\pi(s)}$ has specific requirements
$$|\vec{\pi}(s)|_1 = \sum_a \pi(a|s)= 1; \  0\leq \pi(\cdot|s) \leq 1$$
Common method is to produce 'logits' $e_i$ which are turned into probabilities via 'Softmax':
$$\pi(a_i|s) = \frac{\exp(e_{i})}{\sum_{k=1}^{N_{act}} \exp(e_{k})}$$

> In practice we feed a trajectory of states
> $$\vec{s} = \{s_1,\dots,s_N\}^T$$
> and NN (via Softmax) retrieves a matrix 
> $$P:[N \times N_{actions}] = \{\vec{\pi }(s_1),\dots,\vec{\pi }(s_N)\}^T$$ 
> each prob dist for a state can be sampled.

> Additionally we will define logP for KL divergence
> $$\log P = \{\log \vec{\pi }(s_1),\dots,\log \vec{\pi }(s_N)\}^T= \log\begin{bmatrix} \pi(a_1,s_1) & \pi(a_2,s_1) \\ \pi(a_1,s_2) & \pi(a_2,s_2) \end{bmatrix}$$ 
***

#### Sampling actions

Sampling actions 
$$a_i\sim \pi(\cdot|s_i)$$
can be done via categorical distribution.

>In pytorch we can use <br>`torch.distributions.categorical.Categorical(probs=None, logits=None, validate_args=None)`, </br> which accepts input in either form.

> We sample $P$ for what action policy _thinks is the best_ based on current NN parameters $\theta$
> $$\vec{a}_\pi = \{a_{\pi,1},\dots,a_{\pi,N}\}^T$$
> We also feed list of actions that previous version of NN took at steps $\vec{s}$
> $$\vec{a}_{old} = \{a_{old, 1},\dots,a_{old, N}\}^T$$
> We extract new probabilities from $P$ that correspond to $\vec{a}_\pi$ and $\vec{a}_{old}$
> $$\vec{\pi}= \{\pi(a_{\pi, 1}, s_1), \dots, \pi(a_{\pi, N}, s_N)\}$$
> $$\vec{\pi}_{old} = \{\pi(a_{old, 1}, s_1), \dots, \pi(a_{old, N}, s_N)\}$$
> And for convenience in log-form
> $$ \log \vec{\pi}\ ; \ \log \vec{\pi}_{old}$$

#### Calculating importance sampling ratio

$$L_{\theta_{old}}(\theta) =  \mathbb{E}_{\tau \sim \pi_{old}}\left[\sum_{t}  \frac{\pi(a_t|s_t)}{\pi_{old}(a_t|s_t)} A_{\pi_{old}}(s_t,a_t)\right]$$

> We calculate
>$$\rho_t = \frac{\pi(a_t|s_t)}{\pi_{old}(a_t|s_t)}$$
>by
>$$ \vec{\rho} = \exp \left(\log \vec{\pi}-\log \vec{\pi}_{old}\right) = \exp \left\{log \frac{\pi(a_{\pi, 1})}{\pi(a_{old, 1}, s_1)}, \dots \right\} = \left\{\frac{\pi(a_{\pi, 1})}{\pi(a_{old, 1}, s_1)}, \dots \right\}$$
***

#### KL divergence

Mean KL divergence for discrete case.

Instead of selected probabilities for actions, we have to compare all probabilities. 
> We have computed them previously
$$\log P = \{\log \vec{\pi }(s_1),\dots,\log \vec{\pi }(s_N)\}^T= \log\begin{bmatrix} \pi(a_1,s_1) & \pi(a_2,s_1) \\ \pi(a_1,s_2) & \pi(a_2,s_2) \end{bmatrix}$$
We KL sum divergence over actions in each observed state (axis = -1)
$$D_{KL}(\pi_{old}(\cdot|s_i)||\pi(\cdot|s_i)) =\sum_a \pi_{old}(a|s_i) \bigg(\log \pi_{old}(a|s_i) - \log \pi(a|s_i)\bigg) $$
and take a mean over all states
$$\bar D_{KL}(\pi_{old}||\pi) =   \frac{1}{T}\sum_{i=1}^{T} D_{KL}(\pi_{old}(\cdot|s_i)||\pi(\cdot|s_i))$$

***
***

## Continuous actions (harder)

All implementations for TRPO where action space is continuous parametrize policy as normal distribution $\mathcal{N}_\pi(\mu,\sigma^2)$.
Specifically multivariate diagonal normal distribution.

> Use of classic distributions allows to pre-compute various parameters analytically, which reduces overall computation time.

#### Definition of NN

If our environment receives $N_{act}$ continuous actions, then policy for each state $s$ should produce $N_{act}$ probability distributions, from which individual actions will sampled.
$$\text{Action dist. for state s} = \bigg\{ \mathcal{N}(\mu_{1},\sigma^2_1), \ \dots, \ \mathcal{N}(\mu_{N_{act}},\sigma^2_{N_{act}}) \bigg\}$$
It is the same as defining multivariate gaussian with set of parameters
$$\vec{\mu} = \{ \mu_1, \dots, \mu_{N_{act}} \}$$
$$\vec{\sigma} = \{\sigma_1, \dots, \sigma_{N_{act}} \}$$

> If given a list of $N$ states, we stack vectors of parameters in a larger $N \times N_{acts}$ matrix.

__NOTE:__ It is not necessary to generate individual $\sigma_i$. Each action can have universal parameter.

_Reminder: Of course these parameters are implicitly dependent on weights $\theta$ of NN $\mu_i = \mu_i(\theta)$._
***

#### Sampling actions

Sampling action values for $\mathcal{N}_\pi(\mu,\sigma^2)$ can be done via "reparemetrization trick", used in Variational Autoencoders (VAE) (its used there to enable backprop):
$$\vec{a} \sim \vec{\mu}+ \mathcal{N}_\pi(\vec{0},\vec{1})\cdot\vec{\sigma}$$
***

#### Importance sampling ratio

We will store probabilities in log-space, for stability and convenience:

$$\boxed{\pi(a|s) = \mathcal{N}(a, \sigma^2; \mu) = \frac{1}{\sigma\sqrt{2\pi}}e^{-\frac{1}{2}\left(\frac{a-\mu}{\sigma}\right)^2}}$$

$$\log \pi(a|s) = -\log (\sigma\sqrt{2\pi})- \frac{1}{2}\left(\frac{a-\mu}{\sigma}\right)^2=  - \log \sigma - \frac{1}{2} \log(2 \pi)- \frac{1}{2}\left(\frac{a-\mu}{\sigma}\right)^2$$
$$\boxed{\log \pi(a|s)  = -\frac{1}{2}\left(\log(2 \pi) + 2 \log \sigma + \left(\frac{a-\mu}{\sigma}\right)^2\right)}$$
> Reminder: $\mu$ (and $\sigma$, which is universal for actions) parametrize state. Main distribution argument is action $x_i = a_i$.<br>

Analogous to discrete case we do
$$\rho_t = \frac{\pi(a_t|s_t)}{\pi_{old}(a_t|s_t)} = \exp (\log\pi(a_t|s_t) - \log \pi_{old}(a_t|s_t))$$
> Make use of broadcasting to compute all ratios at once
***

#### KL divergence

Kl divergence can be calculated analytically for two normal multivariate diagonal distributions
([Gaussian_Properties.ipynb](../../Statistics/Gaussian_Properties.ipynb))
$$\boxed{D_{KL}\left(\mathcal{N}_\pi(\sigma^2_1; \mu_1)||\mathcal{N}_\pi(\sigma^2_2; \mu_2)\right) = \frac{1}{2}\left(\frac{(\mu_1 -\mu_2)^2 +\sigma_1^2}{\sigma_2^2} - 1\right) + \log \sigma_2 - \log \sigma_1}$$
We compute sum over actions
$$D_{KL}(\pi(\cdot|s_i)||\pi_{old}(\cdot|s_i)) = \sum_{i\in \vec{a}} \frac{1}{2}\left(\frac{(\vec{\mu_1}_i-\vec{\mu_2}_i)^2 +\vec{\sigma_1}_i^2}{\vec{\sigma_2}_i^2} - 1\right) + \log \vec{\sigma_2 }_i- \log \vec{\sigma_1}_i$$
> Implementation:<br>
> because we store everything in a matrix, KL divergence boils down to application of this function to a matrix, summing over actions an taking mean over remaining dimension.

<style>
img {
  display: block;
  margin-left: auto;
  margin-right: auto;
}
</style>

<img  src="https://spinningup.openai.com/en/latest/_images/math/5808864ea60ebc3702704717d9f4c3773c90540d.svg" 
      width=560 
      style="background-color:white"/>